# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load:
file_to_load='../output_data/cities.csv' # csv file created in part I of homework 

# Read csv cities file and store into data frame:
city_data_df=pd.read_csv(file_to_load)

# Display Result
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapaa,22.0752,-159.3190,26.99,77,40,6.17,US,1651608887
1,1,barrow,71.2906,-156.7887,-5.99,86,100,5.14,US,1651608707
2,2,busselton,-33.6500,115.3333,13.33,68,84,4.85,AU,1651608889
3,3,mar del plata,-38.0023,-57.5575,16.08,59,0,6.17,AR,1651608890
4,4,vaini,-21.2000,-175.2000,23.09,88,20,2.57,TO,1651608890


In [3]:
#Display result 
city_data_df.count()

City_ID       551
City          551
Lat           551
Lng           551
Max Temp      551
Humidity      551
Cloudiness    551
Wind Speed    551
Country       548
Date          551
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Find max humidity level:
max_hum=city_data_df['Humidity'].max()
max_hum

100

In [5]:
# Configure gmaps using API key:
gmaps.configure(api_key=g_key)

In [6]:
# Record 'Lat' and 'Lng' into  locations and drop all values with NaN Values: 
city_data_df=city_data_df.dropna() 
locations=city_data_df[['Lat', 'Lng']].astype(float)

# Convert humidity level to float:
humidity_rate=city_data_df['Humidity'].astype(float)

In [7]:
# Add humity tracking on map:
fig=gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=max_hum,
                                 point_radius=3)

#Display Result
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Create new datafame with parameters and remove NaN:
city_data_narrow=city_data_df[(city_data_df['Max Temp'] < 23) &
                             (city_data_df['Max Temp'] > 18) &
                             (city_data_df['Wind Speed'] < 5) &
                             (city_data_df['Cloudiness'] < 30) &
                             (city_data_df['Humidity'] < 60)]
city_data_narrow=city_data_narrow.dropna() 

# Display Result
city_data_narrow.count()

City_ID       8
City          8
Lat           8
Lng           8
Max Temp      8
Humidity      8
Cloudiness    8
Wind Speed    8
Country       8
Date          8
dtype: int64

In [9]:
# Display Result
city_data_narrow.head(6)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
109,109,faya,18.3851,42.4509,19.06,51,22,1.88,SA,1651608956
111,111,pueblo,38.2545,-104.6091,19.88,41,0,4.63,US,1651608920
175,175,bati,11.1833,40.0167,22.97,38,4,2.85,ET,1651608999
233,233,santiago del estero,-27.7951,-64.2615,21.52,31,0,1.49,AR,1651608787
253,253,mercedes,-34.6515,-59.4307,18.25,41,0,0.12,AR,1651609043
310,310,jiuquan,39.7432,98.5174,18.25,9,0,3.86,CN,1651609074


In [10]:
# Set up lists to hold response info:
name=[]         # Hotel
city=[]         # Hotel in city
country=[]      # Hotel in Country
lat=[]          # Hotel against latitude
lng=[]          # Hotel against longitude

# Loop through data and collate information:

count=1

print('Beginning Data Retrieval')     
print('-----------------------------')
for i in range(len(city_data_narrow)) : 
   
    lati=str(city_data_narrow.iloc[i, 3])    
    lngi=str(city_data_narrow.iloc[i, 4])
    
    target_coordinates=lati + ', ' + lngi  
    target_search='Hotel'
    target_radius=5000
    target_type='lodging'

    # Set dictionary:
    params={
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': g_key
    }

    # Base url:
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # Use disctionary to process request:
    response=requests.get(base_url, params=params)
    
    
    # Convert response to json:
    places_data=response.json()
    
    
    # Print user feedback: 
    print('Looking for Hotels near: ' + str(city_data_narrow.iloc[i, 0]) + ' | Record ' + str(count) + ' of Total ' + str(len(city_data_narrow)))
     
    # Add requested data to  result lists:
    try:                                             
        name.append(places_data['results'][0]['name'])
        city.append(city_data_narrow.iloc[i, 0])
        country.append(city_data_narrow.iloc[i, 1])
        lat.append(places_data['results'][0]['geometry']['location']['lat'])
        lng.append(places_data['results'][0]['geometry']['location']['lng'])
    except IndexError:
        print('No Hotel Near Location... Skipping...') 
  
    count=count+1
    
# Print Ending Log Statement:
print('-------------------------------')
print('Data Retrieval Complete')
print('-------------------------------')

Beginning Data Retrieval
-----------------------------
Looking for Hotels near: 109 | Record 1 of Total 8
Looking for Hotels near: 111 | Record 2 of Total 8
No Hotel Near Location... Skipping...
Looking for Hotels near: 175 | Record 3 of Total 8
No Hotel Near Location... Skipping...
Looking for Hotels near: 233 | Record 4 of Total 8
No Hotel Near Location... Skipping...
Looking for Hotels near: 253 | Record 5 of Total 8
No Hotel Near Location... Skipping...
Looking for Hotels near: 310 | Record 6 of Total 8
No Hotel Near Location... Skipping...
Looking for Hotels near: 332 | Record 7 of Total 8
No Hotel Near Location... Skipping...
Looking for Hotels near: 346 | Record 8 of Total 8
No Hotel Near Location... Skipping...
-------------------------------
Data Retrieval Complete
-------------------------------


In [11]:
# Create a data frame:
hotel_dict={
    'City': city,
    'Country': country,
    'Lat': lat,
    'Lng': lng,
    'Hotel Name': name
}
hotel_df=pd.DataFrame(hotel_dict)

# Display Result
hotel_df.count()

City          1
Country       1
Lat           1
Lng           1
Hotel Name    1
dtype: int64

In [12]:
# Display Result
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,109,faya,42.459555,19.115314,Komanska Utjeha


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Using the template add the hotel marks to the heatmap:
info_box_template="""
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row:
hotel_info=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations=hotel_df[['Lat', 'Lng']]

In [14]:
# Customize figures:
figure_layout={
    'width': '955px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig=gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable:
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map:
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [15]:
# Create a combined map:
fig=gmaps.figure()

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))